# 🧪 การทดสอบ API ทั้งหมด - PaiNaiDee
# 🧪 Complete API Testing - PaiNaiDee

ในโน้ตบุ๊คนี้จะทดสอบการทำงานของ API ทั้งหมดในแอปพลิเคชัน PaiNaiDee

This notebook tests all API endpoints in the PaiNaiDee application

## 📋 รายการ API ที่ทดสอบ / Tested APIs:
- ✅ `/api/talk` - AI Chat API
- ✅ `/api/attractions` - ค้นหาสถานที่ท่องเที่ยว / Search attractions
- ✅ `/api/attractions/<id>` - รายละเอียดสถานที่ / Attraction details
- ✅ `/api/videos` - จัดการวิดีโอ / Video management
- ✅ `/api/videos/upload` - อัปโหลดวิดีโอ / Video upload
- ✅ `/api/user/profile` - โปรไฟล์ผู้ใช้ / User profile
- ✅ `/api/user/favorites` - รายการโปรด / User favorites

## 🛠️ การตั้งค่าเริ่มต้น / Initial Setup

### ติดตั้ง dependencies / Install dependencies

In [ ]:
# ติดตั้ง packages ที่จำเป็น / Install required packages
!pip install requests pandas json5 tabulate

import requests
import json
import pandas as pd
from datetime import datetime
from tabulate import tabulate
import time

### ตั้งค่า API endpoint / Configure API endpoint

In [ ]:
# กำหนด URL ของ API (เปลี่ยนเป็น URL ที่ deploy จริง)
# Set your deployed API URL here (change to your actual deployed URL)

# ตัวอย่าง URLs / Example URLs:
# API_BASE_URL = "https://your-vercel-app.vercel.app/api"  # Vercel deployment
# API_BASE_URL = "https://your-app.hf.space/api"          # Hugging Face Spaces
# API_BASE_URL = "http://localhost:5000/api"              # Local development

API_BASE_URL = "https://your-vercel-app.vercel.app/api"  # 🔧 เปลี่ยนเป็น URL ของคุณ / Change to your URL

print(f"🌐 Testing API at: {API_BASE_URL}")
print(f"🌐 กำลังทดสอบ API ที่: {API_BASE_URL}")

### ฟังก์ชันช่วยเหลือ / Helper Functions

In [ ]:
def call_api(method, endpoint, payload=None, headers=None):
    """
    ฟังก์ชันสำหรับเรียก API
    Helper function to call API endpoints
    
    Args:
        method (str): HTTP method (GET, POST, PUT, DELETE)
        endpoint (str): API endpoint path
        payload (dict): Request payload for POST/PUT
        headers (dict): Request headers
    
    Returns:
        dict: API response data
    """
    url = f"{API_BASE_URL}{endpoint}"
    
    # Default headers
    default_headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    if headers:
        default_headers.update(headers)
    
    try:
        print(f"🔄 {method} {url}")
        
        if method.upper() == 'GET':
            response = requests.get(url, headers=default_headers, timeout=30)
        elif method.upper() == 'POST':
            response = requests.post(url, json=payload, headers=default_headers, timeout=30)
        elif method.upper() == 'PUT':
            response = requests.put(url, json=payload, headers=default_headers, timeout=30)
        elif method.upper() == 'DELETE':
            response = requests.delete(url, headers=default_headers, timeout=30)
        else:
            raise ValueError(f"Unsupported method: {method}")
        
        print(f"✅ Status: {response.status_code}")
        
        if response.status_code < 400:
            try:
                return {
                    'success': True,
                    'status_code': response.status_code,
                    'data': response.json()
                }
            except json.JSONDecodeError:
                return {
                    'success': True,
                    'status_code': response.status_code,
                    'data': response.text
                }
        else:
            return {
                'success': False,
                'status_code': response.status_code,
                'error': response.text
            }
    
    except requests.RequestException as e:
        print(f"❌ Error: {str(e)}")
        return {
            'success': False,
            'error': str(e)
        }

def print_response(response, title="Response"):
    """
    แสดงผลลัพธ์ของ API ในรูปแบบที่อ่านง่าย
    Print API response in a readable format
    """
    print(f"\n{'='*50}")
    print(f"📊 {title}")
    print(f"{'='*50}")
    
    if response.get('success'):
        print(f"✅ สำเร็จ / Success: {response['status_code']}")
        if 'data' in response:
            print(f"📄 Data:")
            print(json.dumps(response['data'], indent=2, ensure_ascii=False))
    else:
        print(f"❌ ล้มเหลว / Failed: {response.get('status_code', 'N/A')}")
        print(f"🚨 Error: {response.get('error', 'Unknown error')}")
    
    print(f"{'='*50}\n")

# กำหนดสีสำหรับผลลัพธ์ / Set colors for results
class Colors:
    GREEN = '\033[92m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    END = '\033[0m'

print("🎉 Helper functions loaded successfully!")
print("🎉 โหลดฟังก์ชันช่วยเหลือเสร็จแล้ว!")

## 🤖 ทดสอบ AI Chat API
## 🤖 Test AI Chat API

In [ ]:
# ทดสอบ AI Chat API
# Test AI Chat API

print("🤖 Testing AI Chat API...")
print("🤖 กำลังทดสอบ AI Chat API...")

# ข้อความทดสอบ / Test messages
test_messages = [
    {
        "message": "สวัสดีครับ แนะนำสถานที่ท่องเที่ยวในกรุงเทพหน่อย",
        "description": "Thai greeting and asking for Bangkok attractions"
    },
    {
        "message": "Hello, can you recommend some beaches in Thailand?",
        "description": "English request for beach recommendations"
    }
]

for i, test_msg in enumerate(test_messages, 1):
    print(f"\n🧪 Test {i}: {test_msg['description']}")
    
    payload = {
        "message": test_msg["message"],
        "userId": "test-user-123",
        "conversationId": f"test-conv-{i}"
    }
    
    response = call_api("POST", "/talk", payload)
    print_response(response, f"AI Chat Test {i}")
    
    time.sleep(1)  # หน่วงเวลาเล็กน้อย / Small delay

## 🏛️ ทดสอบ Attractions API
## 🏛️ Test Attractions API

In [ ]:
# ทดสอบการค้นหาสถานที่ท่องเที่ยว
# Test attraction search

print("🏛️ Testing Attractions API...")
print("🏛️ กำลังทดสอบ Attractions API...")

# ค้นหาสถานที่ท่องเที่ยวทั้งหมด / Search all attractions
print("\n📍 1. Getting all attractions...")
response = call_api("GET", "/attractions")
print_response(response, "All Attractions")

# เก็บ ID ของสถานที่แรกสำหรับทดสอบ detail / Store first attraction ID for detail testing
attraction_id = None
if response.get('success') and response.get('data'):
    attractions = response['data']
    if isinstance(attractions, list) and len(attractions) > 0:
        attraction_id = attractions[0].get('id')
    elif isinstance(attractions, dict) and 'results' in attractions:
        results = attractions['results']
        if len(results) > 0:
            attraction_id = results[0].get('id')

# ค้นหาด้วยคำค้น / Search with query
search_terms = ["temple", "วัด", "beach", "ชายหาด"]

for i, term in enumerate(search_terms, 2):
    print(f"\n🔍 {i}. Searching for '{term}'...")
    response = call_api("GET", f"/attractions?search={term}")
    print_response(response, f"Search: {term}")
    time.sleep(0.5)

# ค้นหาตามหมวดหมู่ / Search by category
categories = ["temple", "beach", "mountain", "food"]

for i, category in enumerate(categories, len(search_terms) + 2):
    print(f"\n🏷️ {i}. Getting {category} attractions...")
    response = call_api("GET", f"/attractions?category={category}")
    print_response(response, f"Category: {category}")
    time.sleep(0.5)

## 📍 ทดสอบ Attraction Detail API
## 📍 Test Attraction Detail API

In [ ]:
# ทดสอบการดูรายละเอียดสถานที่ท่องเที่ยว
# Test attraction detail

print("📍 Testing Attraction Detail API...")
print("📍 กำลังทดสอบ Attraction Detail API...")

if attraction_id:
    print(f"\n🏛️ Getting details for attraction ID: {attraction_id}")
    response = call_api("GET", f"/attractions/{attraction_id}")
    print_response(response, f"Attraction Detail: {attraction_id}")
else:
    print("⚠️ No attraction ID available from previous test")
    print("⚠️ ไม่มี attraction ID จากการทดสอบก่อนหน้า")
    
    # ลองใช้ ID ตัวอย่าง / Try with sample IDs
    sample_ids = ["1", "temple-1", "beach-1", "wat-pho"]
    
    for sample_id in sample_ids:
        print(f"\n🧪 Trying sample ID: {sample_id}")
        response = call_api("GET", f"/attractions/{sample_id}")
        print_response(response, f"Sample ID: {sample_id}")
        
        if response.get('success'):
            break
        time.sleep(0.5)

## 🎥 ทดสอบ Videos API
## 🎥 Test Videos API

In [ ]:
# ทดสอบ Videos API
# Test Videos API

print("🎥 Testing Videos API...")
print("🎥 กำลังทดสอบ Videos API...")

# ดูรายการวิดีโอทั้งหมด / Get all videos
print("\n📹 1. Getting all videos...")
response = call_api("GET", "/videos")
print_response(response, "All Videos")

# ค้นหาวิดีโอ / Search videos
search_terms = ["travel", "thailand", "tour"]

for i, term in enumerate(search_terms, 2):
    print(f"\n🔍 {i}. Searching videos for '{term}'...")
    response = call_api("GET", f"/videos?search={term}")
    print_response(response, f"Video Search: {term}")
    time.sleep(0.5)

# ดูวิดีโอตามหมวดหมู่ / Get videos by category
categories = ["attraction", "food", "culture"]

for i, category in enumerate(categories, len(search_terms) + 2):
    print(f"\n🏷️ {i}. Getting {category} videos...")
    response = call_api("GET", f"/videos?category={category}")
    print_response(response, f"Video Category: {category}")
    time.sleep(0.5)

## ⬆️ ทดสอบ Video Upload API
## ⬆️ Test Video Upload API

In [ ]:
# ทดสอบการอัปโหลดวิดีโอ (จำลอง)
# Test video upload (simulation)

print("⬆️ Testing Video Upload API...")
print("⬆️ กำลังทดสอบ Video Upload API...")

# จำลองข้อมูลการอัปโหลดวิดีโอ / Simulate video upload data
upload_data = {
    "title": "สวยงามของวัดพระแก้ว",
    "title_en": "Beauty of Wat Phra Kaew",
    "description": "วิดีโอแสดงความสวยงามของวัดพระแก้วในกรุงเทพมหานคร",
    "description_en": "Video showcasing the beauty of Wat Phra Kaew in Bangkok",
    "category": "attraction",
    "tags": ["temple", "bangkok", "culture", "วัด", "กรุงเทพ"],
    "duration": 120,
    "file_size": 1024000,
    "file_name": "wat_phra_kaew_tour.mp4",
    "uploaded_by": "test-user-123"
}

print("\n📤 1. Uploading video metadata...")
response = call_api("POST", "/videos/upload", upload_data)
print_response(response, "Video Upload")

# ทดสอบการอัปโหลดแบบ multipart (จำลอง) / Test multipart upload (simulation)
print("\n📎 2. Testing multipart upload simulation...")

multipart_data = {
    "title": "ตลาดน้ำดำเนินสะดวก",
    "title_en": "Damnoen Saduak Floating Market",
    "category": "market",
    "file_info": {
        "filename": "floating_market.mp4",
        "content_type": "video/mp4",
        "size": 2048000
    },
    "chunks_total": 4,
    "chunk_size": 512000
}

response = call_api("POST", "/videos/upload", multipart_data)
print_response(response, "Multipart Upload")

## 👤 ทดสอบ User Profile API
## 👤 Test User Profile API

In [ ]:
# ทดสอบ User Profile API
# Test User Profile API

print("👤 Testing User Profile API...")
print("👤 กำลังทดสอบ User Profile API...")

# กำหนด headers สำหรับ authentication (จำลอง) / Set headers for authentication (simulation)
auth_headers = {
    "Authorization": "Bearer test-token-123",
    "X-User-ID": "test-user-123"
}

# ดูโปรไฟล์ผู้ใช้ / Get user profile
print("\n👤 1. Getting user profile...")
response = call_api("GET", "/user/profile", headers=auth_headers)
print_response(response, "User Profile")

# อัปเดตโปรไฟล์ผู้ใช้ / Update user profile
print("\n✏️ 2. Updating user profile...")

profile_update = {
    "name": "ทดสอบ ผู้ใช้งาน",
    "name_en": "Test User",
    "email": "test@example.com",
    "preferences": {
        "language": "th",
        "favorite_categories": ["temple", "beach", "food"],
        "notifications": True,
        "newsletter": False
    },
    "location": {
        "province": "กรุงเทพมหานคร",
        "country": "Thailand"
    }
}

response = call_api("PUT", "/user/profile", profile_update, auth_headers)
print_response(response, "Profile Update")

# ดูโปรไฟล์หลังการอัปเดต / Get profile after update
print("\n🔄 3. Getting updated profile...")
response = call_api("GET", "/user/profile", headers=auth_headers)
print_response(response, "Updated Profile")

## ❤️ ทดสอบ User Favorites API
## ❤️ Test User Favorites API

In [ ]:
# ทดสอบ User Favorites API
# Test User Favorites API

print("❤️ Testing User Favorites API...")
print("❤️ กำลังทดสอบ User Favorites API...")

# ดูรายการโปรดของผู้ใช้ / Get user favorites
print("\n📋 1. Getting user favorites...")
response = call_api("GET", "/user/favorites", headers=auth_headers)
print_response(response, "User Favorites")

# เพิ่มสถานที่ในรายการโปรด / Add attraction to favorites
favorite_attractions = [
    "wat-pho",
    "grand-palace",
    "chatuchak-market"
]

for i, attraction_id in enumerate(favorite_attractions, 2):
    print(f"\n❤️ {i}. Adding {attraction_id} to favorites...")
    response = call_api("POST", f"/user/favorites/{attraction_id}", headers=auth_headers)
    print_response(response, f"Add Favorite: {attraction_id}")
    time.sleep(0.5)

# ดูรายการโปรดหลังการเพิ่ม / Get favorites after adding
print(f"\n📋 {len(favorite_attractions) + 2}. Getting updated favorites...")
response = call_api("GET", "/user/favorites", headers=auth_headers)
print_response(response, "Updated Favorites")

# ลบสถานที่จากรายการโปรด / Remove attraction from favorites
if favorite_attractions:
    remove_id = favorite_attractions[0]
    print(f"\n💔 {len(favorite_attractions) + 3}. Removing {remove_id} from favorites...")
    response = call_api("DELETE", f"/user/favorites/{remove_id}", headers=auth_headers)
    print_response(response, f"Remove Favorite: {remove_id}")

# ดูรายการโปรดสุดท้าย / Get final favorites
print(f"\n📋 {len(favorite_attractions) + 4}. Getting final favorites...")
response = call_api("GET", "/user/favorites", headers=auth_headers)
print_response(response, "Final Favorites")

## 📊 สรุปผลการทดสอบ / Test Summary
## 📊 Test Results Summary

In [ ]:
# สรุปผลการทดสอบ
# Summarize test results

print("📊 Test Summary / สรุปผลการทดสอบ")
print("=" * 60)

# รายการ API ที่ทดสอบ / List of tested APIs
tested_apis = [
    {
        "endpoint": "/api/talk",
        "description": "AI Chat API",
        "description_th": "API แชทกับ AI",
        "methods": ["POST"]
    },
    {
        "endpoint": "/api/attractions",
        "description": "Search attractions",
        "description_th": "ค้นหาสถานที่ท่องเที่ยว",
        "methods": ["GET"]
    },
    {
        "endpoint": "/api/attractions/<id>",
        "description": "Get attraction details",
        "description_th": "ดูรายละเอียดสถานที่ท่องเที่ยว",
        "methods": ["GET"]
    },
    {
        "endpoint": "/api/videos",
        "description": "Video management",
        "description_th": "จัดการวิดีโอ",
        "methods": ["GET"]
    },
    {
        "endpoint": "/api/videos/upload",
        "description": "Upload videos",
        "description_th": "อัปโหลดวิดีโอ",
        "methods": ["POST"]
    },
    {
        "endpoint": "/api/user/profile",
        "description": "User profile management",
        "description_th": "จัดการโปรไฟล์ผู้ใช้",
        "methods": ["GET", "PUT"]
    },
    {
        "endpoint": "/api/user/favorites",
        "description": "User favorites",
        "description_th": "รายการโปรดของผู้ใช้",
        "methods": ["GET", "POST", "DELETE"]
    }
]

# แสดงตารางสรุป / Display summary table
table_data = []
for api in tested_apis:
    methods_str = ", ".join(api["methods"])
    table_data.append([
        api["endpoint"],
        api["description"],
        api["description_th"],
        methods_str
    ])

headers = ["Endpoint", "Description (EN)", "Description (TH)", "Methods"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

print(f"\n✅ Total APIs tested: {len(tested_apis)}")
print(f"✅ จำนวน API ที่ทดสอบทั้งหมด: {len(tested_apis)}")

print(f"\n🕐 Test completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🕐 ทดสอบเสร็จสิ้นเมื่อ: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n🎉 การทดสอบเสร็จสิ้น! / Testing completed!")
print("\n📝 หมายเหตุ / Note:")
print("- เปลี่ยน API_BASE_URL ให้เป็น URL ที่ deploy จริง / Change API_BASE_URL to your actual deployed URL")
print("- ปรับ authentication headers ตามระบบที่ใช้งานจริง / Adjust authentication headers for your actual system")
print("- API บางตัวอาจต้องการ database หรือ external services / Some APIs may require database or external services")